# Table of contents
1. [Introduction](#intro)
1. [Setup](#setup)
2. [Methods](#methods)
    1. [Original tangent approximation](#org-tangent)
    2. [Adapted tangent approximation](#adapted-tangent)
    3. [Finite difference approximation](#finite-diff)

# Introduction: Differentiation of matrix exp for weighted GNMs <a name="intro"></a>
From: https://www.biorxiv.org/content/10.1101/2023.06.23.546237v1?med=mas.

This is mostly concerned with finding the fastest way to compute the derivative of the matrix exponential.
$$
f(w_{i,j}) = (c_{i,k} * d_{i,j})^{\omega}
$$
Where, the communicability matrix is defined as follows. This captures the proportion of signals, that propagate randomly from node i would reach node j over an infinite time horizon.
$$
c_{i,j} = e^{s^{-1/2}w_{i,j}s^{-1/2}}
$$

Finally, the update rule is given as:
$$
w_{i,j} = w_{i,j} - \alpha f'(w_{i,j})
$$

**Question**<br>
In the code implementation this seems to be bound to zero in Line 110, but there is no upper bound, should it be normalized to range [0,1]?

## Setup <a name="setup"></a>

In [1]:
using Polynomials
using ForwardDiff
using ExponentialUtilities
using LinearAlgebra
using BenchmarkTools
using StatsBase: sample
include("test_data.jl")

load_weight_test_data (generic function with 1 method)

In [2]:
W = [0.0 0.8 0.0;
     0.8 0.0 0.2; 
     0.0 0.2 0.0]
demo_edge = [CartesianIndex(1, 2)]
W

3×3 Matrix{Float64}:
 0.0  0.8  0.0
 0.8  0.0  0.2
 0.0  0.2  0.0

# Methods <a name="methods"></a>
Here we will work on different approaches as to compute the derivative.
In this demonstration, we will simply work on the matrix exponential differentiation, as the rest of the operation is trivial and computationally cheap.

For:
$$
f(W)=e^{W}
$$
Find
$$
\frac{\delta f}{\delta W_{i,j}}
$$
Or more specifically 
$$
f(W)=\sum_{i,j} e^{W}
$$

But this also simply boils down to the evaluation of the derivative at one (a matrix of ones at the same size as W for the jacobian vector product).

## Original tangent approximation <a name="org-tangent"></a>

Here we are computing the derivative for a single edge. We will use the tangent approximation to compute the derivative. 
This is the approach used in the Akarca paper and available in the repo, as can be seen below this will not produce the correct result.

**Problem 1**<br>
As far as I understand, there is a small but significant error in the implementation provided at https://github.com/DanAkarca/weighted_generative_models/blob/main/weighted_generative_model.m in line 197.
When fitting the first-order polynomial, the independent variable should not be the range of the differences to the edge, which have been used, but should be the actual values, the edge is taking on.

**Problem 2**<br>
The second problem is that we will often need to compute the derivative with respect to a value in the matrix, which is currently zero. (This happens easily as the value for any weight has a lower bound of zero which is often hit).
Now the approach implemented in the code builds a range of x value by using a value for the number of repititions and resolutions for the tangent approximations, but it multiples the resolution with the current x-value, which will produce a vector of zeros whenever the current value is zero, which will always produce a zero slope and zero derivative approximations - which is wrong.

**Problem 2**
In the same implementation the derivative is approximated from the tangent approximation after nudging the edge (i,j) as well as (j,i).
In a sense we are computing the derivative with respect to both (i,j) and (j,i) at once.
First of all, this will create computational overhead, as the derivatives (i.e., the jacobian) will also always be symmetric.
Secondly, this will create a problem when we are trying to compute the gradient, which we currently simply take as the the derivative.
But then we update both elements in the W matrix with the same gradient which is the derivative with respect to nudging both (i,j) and (j,i) - thus our update to the objective function is overshooting.

See line 208 and 208.

In [3]:
function paper_tangent_approx(W, edges::Vector{CartesianIndex{2}}, 
    resolution = 0.01, steps = 5)::Vector{Float64}
    results = zeros(length(edges))
    rep_vec = collect(range(-steps*resolution, steps*resolution, step=resolution))

    for (i_edge, edge_idx) in enumerate(edges)
        
        # Points for evaluation
        edge_val = W[edge_idx]
        reps = [edge_val * (1 + i) for i in rep_vec]

        # For each nudge save difference in communicability 
        sum_comm = zeros(length(reps))
        for (i_rep, rep) in enumerate(reps)
            W_copy = copy(W)
            W_copy[edge_idx] = W_copy[edge_idx[2], edge_idx[1]] = rep
            comm = exp(W_copy)
            sum_comm[i_rep] = sum(comm)
        end

        # Line 197 in MATLAB code
        x = 1:length(reps)
        results[i_edge] = fit(x, sum_comm, 1)[1]
    end

    return results
end

paper_tangent_approx(W, demo_edge, 0.01), paper_tangent_approx(W, demo_edge, 0.2)

([0.03861909629029788], [0.830380354061412])

## Adapted tangent approximation <a name="adapted-tangent"></a>

In [4]:
function tangent_approx(f::Function, W::Matrix{Float64}, edges::Vector{CartesianIndex{2}}, 
    resolution = 0.01, steps = 5)::Vector{Float64}
    results = zeros(length(edges))
    rep_vec = collect(range(-steps*resolution, steps*resolution, step=resolution))

    for (i_edge, edge_idx) in enumerate(edges)     
        # Points for evaluation
        edge_val = W[edge_idx]
        reps = [edge_val + sign(edge_val) * min(abs(edge_val), 1e-3) * i for i in rep_vec]
        println(reps)

        # For each nudge save difference in communicability 
        sum_comm = zeros(length(reps))
        for (i_rep, rep) in enumerate(reps)
            W_copy = copy(W)
            W_copy[edge_idx] = rep 
            sum_comm[i_rep] = f(W_copy, edge_idx)
        end

        results[i_edge] = fit(reps, sum_comm, 1)[1]
    end

    return results
end

f = (W, _) -> sum(exp(W))
tangent_approx(f, W, demo_edge, 0.01), tangent_approx(f, W, demo_edge, 0.1)

([2.4132511356664494], [2.4132511369955423])

## Finite difference method <a name="finite-diff"></a>
Compute the approximate derivative using the finite difference method.
At the moment, this is a significantly faster but less accurate method than the tangent approximation.
Only implemented to be potentially extended to complex step approximation.

References:<br>
https://en.wikipedia.org/wiki/Finite_difference_method

In [5]:
function finite_diff(f, W, edges::Vector{CartesianIndex{2}}, delta::Float64)::Vector{Float64}
    results = zeros(length(edges))
    for (i_edge, edge_idx) in enumerate(edges) 
        # Evaluate the function at two nearby points
        W_copy = copy(W)
        W_copy[edge_idx] = W_copy[edge_idx] + delta
        f_plus_delta = sum(exp(W_copy))
        
        W_copy[edge_idx]  = W_copy[edge_idx] - 2*delta
        f_minus_delta = sum(exp(W_copy))

        # Calculate the derivative approximation
        results[i_edge] = (f_plus_delta - f_minus_delta) / (2 * delta)
    end

    return results
end

f = W -> sum(exp(W))
finite_diff(f, W, demo_edge, 0.01), finite_diff(f, W, demo_edge,0.2)

([2.413251884607348], [2.4135507160903513])

## Forward Differentiation
Results are derived in the numerator layout, which means that the dimensionality of the input is added to the right

In [6]:
function forward_diff_j(W::Matrix{Float64}, edges::Vector{CartesianIndex{2}})::Vector{Float64}
    # Column indices for retrieval
    indices = collect(CartesianIndices(W))
    index_vec = sort(vec(indices), by = x -> x[1])

    diff_exp(W) = exponential!(copyto!(similar(W), W), ExpMethodGeneric())
    J = ForwardDiff.jacobian(diff_exp, W)

    results = zeros(length(edges))
    tangent = vec(permutedims(exp(W), [2, 1]))
    for (i_edge, edge) in enumerate(edges)
        # we get all partial derivative positions that are non-zero
        Jₓ = J[:, findfirst(x -> x == edge, index_vec)]
        results[i_edge] = sum(Jₓ)
    end

    return results
end

forward_diff_j(W, demo_edge)

1-element Vector{Float64}:
 2.4132511356618336

## Forward Differentiation with Jacobian vector product

In [7]:
function forward_diff_jvp(W::Matrix{Float64}, edges::Vector{CartesianIndex{2}})::Vector{Float64}
    tangent = ones(size(W))
    diff_exp(W) = exponential!(copyto!(similar(W), W), ExpMethodGeneric())
    g(t) = diff_exp(W + t * tangent)
    JVP = ForwardDiff.derivative(g, 0.0)
    return JVP[edges]
end

forward_diff_jvp(W, demo_edge)

1-element Vector{Float64}:
 2.413251135661834

## Fréchet: Block enlarge

In [8]:
function frechet_block_enlarge(W::Matrix{Float64}, edges::Vector{CartesianIndex{2}})::Vector{Float64}
    E = ones(size(W))
    n = size(W, 1)
    M = [W E; zeros(size(W)) W]
    expm_M = exp(M) 
    frechet_AE = expm_M[1:n, n+1:end]
    return frechet_AE[edges]
end

frechet_block_enlarge(W, demo_edge)

1-element Vector{Float64}:
 2.413251135661833

## Fréchet: Scaling-Pade-Squaring

The algorithm:
$$
\begin{array}{l}
\text{for } m = [3, 5, 7, 9] \\
\quad \quad \text{if } ||A||_{1} \leq \theta_{m} \\
\quad \quad \quad \quad r_{m}(A) \quad \text{ \textit{form Pade approximant to A}} \\
\quad \quad \quad \quad U, V     \quad \text{ \textit{Evaluate U and V, see below for }} r_{13} \\
\quad \quad \quad \quad s=0 \\
\quad \quad \quad \quad   \text{break} \\
\quad \quad \text{end} \\
\text{end} \\

\text{if } ||A||_{1} \geq  \theta_{m[-1]} \\
\quad \quad s = \text{ceil}(\log_{2}(||A||_{1}/ \theta_{13})) \\
\quad \quad A = A/2^{s} \\
\quad \quad A_{2} = A^{2}, A_{4} = A_{2}^{2}, A_{6} = A_{2}A_{4}\\ 
\quad \quad U = A[A_{6}(b_{13} A_{6} + b_{11} A_{4} + b_{9} A_{2}) + b_{7} A_{6} + b_{5} A_{4} + b_{3} A_{2} + b_{1} I]\\
\quad \quad V = A_{6}(b_{12}A_{6} + b_{10}A_{4} + b_{8}A_{2}) + b_{6}A_{6} + b_{4}A_{4} + b_{2}A_{2} + b_{0}I\\
\text{end} \\

\text{Solve } (-U+V)r_{m}(A) = U + V \text{ for } r_{m} \\
\text{for } k = 1:s \\
\quad \quad r_{m} = r_{m}*r_{m} \\
\text{end} \\
\text{return } r_{m} \\
\end{array}
$$

References:<br>
Higham (2008), Functions of Matrices - Theory and Computation", Chapter 10, Algorithm 10.20 <br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.expm_frechet.html#scipy.linalg.expm_frechet <br>
https://rdrr.io/cran/expm/man/expmFrechet.html

In [9]:
function _diff_pade3(A, E, ident)
    b = (120.0, 60.0, 12.0, 1.0)
    A2 = A * A
    M2 = A * E + E * A
    U = A * (b[4] * A2 + b[2] * ident)
    V = b[3] * A2 + b[1] * ident
    Lu = A * (b[4] * M2) + E * (b[4] * A2 + b[2] * ident)
    Lv = b[3] * M2
    return U, V, Lu, Lv
end
    
function _diff_pade5(A, E, ident)
    b = (30240.0, 15120.0, 3360.0, 420.0, 30.0, 1.0)
    A2 = A * A
    M2 = A * E + E * A
    A4 = A2 * A2
    M4 = A2 * M2 + M2 * A2
    U = A * (b[6] * A4 + b[4] * A2 + b[2] * ident)
    V = b[5] * A4 + b[3] * A2 + b[1] * ident
    Lu = A * (b[6] * M4 + b[4] * M2) + E * (b[6] * A4 + b[4] * A2 + b[2] * ident)
    Lv = b[5] * M4 + b[3] * M2
    return U, V, Lu, Lv
end

function _diff_pade7(A, E, ident)
    b = (17297280.0, 8648640.0, 1995840.0, 277200.0, 25200.0, 1512.0, 56.0, 1.0)
    A2 = A * A
    M2 = A * E + E * A
    A4 = A2 * A2
    M4 = A2 * M2 + M2 * A2
    A6 = A2 * A4
    M6 = A4 * M2 + M4 * A2
    U = A * (b[8] * A6 + b[6] * A4 + b[4] * A2 + b[2] * ident)
    V = b[7] * A6 + b[5] * A4 + b[3] * A2 + b[1] * ident
    Lu = A * (b[8] * M6 + b[6] * M4 + b[4] * M2) + E * (b[8] * A6 + b[6] * A4 + b[4] * A2 + b[2] * ident)
    Lv = b[7] * M6 + b[5] * M4 + b[3] * M2
    return U, V, Lu, Lv
end

function _diff_pade9(A, E, ident)
    b = (17643225600.0, 8821612800.0, 2075673600.0, 302702400.0, 30270240.0, 
         2162160.0, 110880.0, 3960.0, 90.0, 1.0)
    A2 = A * A
    M2 = A * E + E * A
    A4 = A2 * A2
    M4 = A2 * M2 + M2 * A2
    A6 = A2 * A4
    M6 = A4 * M2 + M4 * A2
    A8 = A4 * A4
    M8 = A4 * M4 + M4 * A4
    U = A * (b[10] * A8 + b[8] * A6 + b[6] * A4 + b[4] * A2 + b[2] * ident)
    V = b[9] * A8 + b[7] * A6 + b[5] * A4 + b[3] * A2 + b[1] * ident
    Lu = A * (b[10] * M8 + b[8] * M6 + b[6] * M4 + b[4] * M2) + E * (b[10] * A8 + b[8] * A6 + b[6] * A4 + b[4] * A2 + b[2] * ident)
    Lv = b[9] * M8 + b[7] * M6 + b[5] * M4 + b[3] * M2
    return U, V, Lu, Lv
end

function _diff_pade13(A, E, ident)
    # pade order 13
    A2 = A * A
    M2 = A * E + E * A
    A4 = A2 * A2
    M4 = A2 * M2 + M2 * A2
    A6 = A2 * A4
    M6 = A4 * M2 + M4 * A2
    b = (64764752532480000., 32382376266240000., 7771770303897600.,
            1187353796428800., 129060195264000., 10559470521600.,
            670442572800., 33522128640., 1323241920., 40840800., 960960.,
            16380., 182., 1.)
    W1 = b[14] * A6 + b[12] * A4 + b[10] * A2
    W2 = b[8] * A6 + b[6] * A4 + b[4] * A2 + b[2] * ident
    Z1 = b[13] * A6 + b[11] * A4 + b[9] * A2
    Z2 = b[7] * A6 + b[5] * A4 + b[3] * A2 + b[1] * ident
    W = A6 * W1 + W2
    U = A * W
    V = A6 * Z1 + Z2
    Lw1 = b[14] * M6 + b[12] * M4 + b[10] * M2
    Lw2 = b[8] * M6 + b[6] * M4 + b[4] * M2
    Lz1 = b[13] * M6 + b[11] * M4 + b[9] * M2
    Lz2 = b[7] * M6 + b[5] * M4 + b[3] * M2
    Lw = A6 * Lw1 + M6 * W1 + Lw2
    Lu = A * Lw + E * W
    Lv = A6 * Lz1 + M6 * Z1 + Lz2
    return U, V, Lu, Lv
end

ell_table_61 = (nothing, 2.11e-8, 3.56e-4, 1.08e-2, 6.49e-2, 2.00e-1, 4.37e-1,
        7.83e-1, 1.23e0, 1.78e0, 2.42e0, 3.13e0, 3.90e0, 4.74e0, 5.63e0,
        6.56e0, 7.52e0, 8.53e0, 9.56e0, 1.06e1, 1.17e1);

In [10]:
function frechet_algo(A::Matrix{Float64}, edges::Vector{CartesianIndex{2}})::Vector{Float64}
    E = ones(size(A))
    n = size(A, 1)
    s = nothing
    ident = Matrix{Float64}(I, n, n)
    A_norm_1 = norm(A, 1)
    m_pade_pairs = [(3, _diff_pade3),(5, _diff_pade5),(7, _diff_pade7),(9, _diff_pade9)]
    for (m, pade) in m_pade_pairs
        if A_norm_1 <= ell_table_61[m]
            U, V, Lu, Lv = pade(A, E, ident)
            s = 0
            break
        end
    end
    if s == nothing
        # scaling
        s = max(0, ceil(Int, log2(A_norm_1 / ell_table_61[13])))
        A *= 2.0^-s
        E *= 2.0^-s
        U, V, Lu, Lv = _diff_pade13(A, E, ident)
    end
    
    # factor once and solve twice
    lu_piv = lu(-U + V)
    R = lu_piv \ (U + V)
    L = lu_piv \ (Lu + Lv + ((Lu - Lv) * R))
    
    # repeated squaring
    for k in 1:s
        L = R * L + L * R
        R= R * R
    end
    return L[edges]
end

frechet_algo(W, demo_edge)

1-element Vector{Float64}:
 2.413251135661833

# Benchmarking <a name="benchmarking"></a>

In [11]:
function init_sparse_matrix(n = 100, density = 0.2)
    # Initialize a sparse matrix
    W = zeros(n, n)
    for i in 1:n, j in 1:n
        if rand() < (density / 2)
            W[i, j] = W[j,i] = rand()
        end
    end
    return W
end

W_bench = init_sparse_matrix(50, 0.10)
edges_bench = findall(x -> x != 0, W_bench)
f = (W, _) -> sum(exp(W))

@btime tangent_approx(f, $W_bench, $edges_bench, 0.01) 
@btime finite_diff(f, $W_bench, $edges_bench, 0.1) 
@btime forward_diff_j($W_bench, $edges_bench) 
@btime forward_diff_jvp($W_bench, $edges_bench) 
@btime frechet_block_enlarge($W_bench, $edges_bench) 
@btime frechet_algo($W_bench, $edges_bench);

  503.245 ms (55913 allocations: 384.22 MiB)


  87.069 ms (7851 allocations: 61.46 MiB)


  1.720 s (5970 allocations: 312.23 MiB)


  1.502 ms (20 allocations: 315.22 KiB)


  3.362 ms (24 allocations: 609.55 KiB)


  412.083 μs (220 allocations: 1.86 MiB)


# Extension to full objective function

In [12]:
# load synthetic data
W_Y, D, A_init = load_weight_test_data()
A_Y = Float64.(W_Y .> 0);
α = 0.01
ω = 0.9
ϵ = 1e-5
m_seed = Int(sum(A_init))
m_all = Int(sum(A_Y))
resolution = 0.01
steps = 5

zero_indices = findall(==(0), triu(A_init, 1))
edges_to_add = sample(zero_indices, m_all-m_seed; replace = false);

In [13]:
function test_tangent(m_max::Int, normalize::Bool)
    A_current = copy(A_init)
    W_current = copy(A_init)

    if normalize
        function f(W, edge_idx)
            node_strengths = dropdims(sum(W, dims=2), dims=2)
            node_strengths[node_strengths.==0] .= ϵ
            norm_fact = sqrt(inv(Diagonal(node_strengths)))
            return (sum(exp(norm_fact * W * norm_fact))  * D[edge_idx])^ω
        end
    else
        f = (W, edge_idx) -> (sum(exp(W))  * D[edge_idx])^ω
    end

    for m in 1:m_max
        # Get the edge
        edge_idx = edges_to_add[m-m_seed]
        rev_idx = CartesianIndex(edge_idx[2], edge_idx[1])
        A_current[edge_idx] = W_current[edge_idx] =  1
        A_current[rev_idx] = W_current[rev_idx] =  1    
        edge_indices = findall(!=(0), triu(A_current, 1))
        
        tangent_d = tangent_approx(f, W_current, edge_indices, resolution, steps)
        println(round.(tangent_d, digits=2))

        for (i_edge, edge) in enumerate(edge_indices)
            W_current[edge] -= (α * tangent_d[i_edge])
            W_current[edge] = max(0, W_current[edge])
            W_current[CartesianIndex(edge[2], edge[1])] = W_current[edge]
        end
    end
    return W_current
end;

In [14]:
function test_frechet_algo(m_max::Int, normalize::Bool)
    A_current = copy(A_init)
    W_current = copy(A_init)

    for m in 1:m_max
        # Get the edge
        edge_idx = edges_to_add[m-m_seed]
        rev_idx = CartesianIndex(edge_idx[2], edge_idx[1])
        A_current[edge_idx] = W_current[edge_idx] =  1
        A_current[rev_idx] = W_current[rev_idx] =  1    
        edge_indices = findall(!=(0), triu(A_current, 1))

        if normalize
            node_strengths = dropdims(sum(W_current, dims=2), dims=2)
            node_strengths[node_strengths.==0] .= ϵ
            norm_fact = sqrt(inv(Diagonal(node_strengths)))
            current_expo = norm_fact * W_current * norm_fact
        else
            current_expo = W_current
        end

        # compute the derivative
        current_Y = sum(exp(current_expo))
        frechet_d = frechet_algo(current_expo, edge_indices)
        frechet_d = [ω * D[edge]^ω  * frechet_d[i_edge] * current_Y^(ω-1) for (i_edge, edge) in enumerate(edge_indices)]
        println(round.(frechet_d, digits=2))

        for (i_edge, edge) in enumerate(edge_indices)
            W_current[edge] -= (α * frechet_d[i_edge])
            W_current[edge] = max(0, W_current[edge])
            W_current[CartesianIndex(edge[2], edge[1])] = W_current[edge]
        end
    end
    return W_current
end;

In [15]:
#@time W_res_frechet = test_frechet_algo(m_max, true);
@time W_res_tangent = test_tangent(3, false);

[142.12]
[75.41, 0.0]


[29.48, 35.45, 0.0]
  0.422913 seconds (350.26 k allocations: 42.911 MiB, 3.30% gc time, 50.22% compilation time)


In [16]:
m_max = 40
#@time W_res_tangent = test_tangent(m_max, );
@time W_res_frechet = test_frechet_algo(m_max, false);
#sum(abs.(W_res_tangent .- W_res_frechet))

[142.12]
[75.41, 52.28]
[29.48, 35.45, 52.24]
[21.91, 27.66, 75.92, 52.14]
[17.6, 27.67, 35.55, 52.16, 28.88]


[14.74, 29.25, 27.64, 27.9, 52.08, 21.6]
[12.71, 21.8, 86.71, 47.43, 47.89, 52.02, 17.39]
[11.21, 17.56, 36.5, 29.57, 29.86, 52.11, 61.8, 14.64]
[10.81, 14.73, 31.94, 27.64, 27.91, 93.43, 125.73, 55.02, 12.64]
[10.85, 12.75, 32.04, 137.75, 27.72, 27.99, 52.25, 39.86, 22.8, 11.17]
[10.85, 11.23, 72.11, 32.06, 50.7, 27.74, 28.01, 52.28, 39.88, 22.81, 10.65]


[10.84, 19.1, 155.51, 58.47, 32.01, 50.63, 27.7, 27.97, 52.21, 39.82, 22.78, 10.63]
[10.85, 10.8, 112.7, 51.39, 26.53, 32.06, 50.71, 27.74, 28.01, 52.28, 39.88, 22.81, 10.65]
[10.85, 10.8, 41.46, 51.39, 26.53, 32.06, 50.71, 27.74, 28.01, 52.28, 39.88, 22.81, 78.18, 10.65]
[10.85, 10.79, 41.43, 51.36, 26.51, 32.04, 50.67, 27.72, 27.99, 81.16, 52.25, 39.85, 22.79, 35.75, 10.64]
[10.85, 10.79, 41.43, 51.36, 26.52, 32.04, 50.68, 27.73, 27.99, 36.05, 89.79, 64.01, 39.86, 22.8, 28.74, 10.64]
[10.84, 18.54, 41.41, 72.37, 51.33, 26.5, 32.02, 50.65, 27.71, 27.98, 29.84, 62.86, 33.73, 39.83, 22.78, 49.36, 10.64]


[10.84, 12.43, 41.42, 35.1, 51.34, 26.51, 55.04, 110.02, 50.66, 27.72, 48.08, 29.85, 52.83, 24.08, 39.84, 22.79, 33.09, 10.64]
[10.85, 70.1, 10.8, 41.46, 26.65, 51.39, 26.53, 32.06, 40.51, 50.71, 27.74, 28.01, 29.88, 52.28, 23.56, 39.88, 22.81, 28.76, 10.65]
[10.84, 34.74, 10.79, 41.42, 26.63, 51.34, 26.51, 32.03, 40.47, 50.66, 27.72, 27.98, 29.85, 52.23, 23.54, 59.77, 39.84, 22.79, 28.73, 10.64]
[10.84, 44.25, 10.79, 73.4, 41.4, 26.62, 51.32, 26.5, 32.02, 40.46, 50.64, 27.7, 27.97, 29.84, 52.21, 23.53, 32.86, 39.83, 22.78, 28.72, 10.63]
[10.84, 29.51, 10.79, 35.24, 41.41, 26.62, 51.33, 26.5, 32.02, 40.46, 50.65, 27.71, 27.98, 29.84, 52.22, 23.53, 104.4, 40.35, 39.83, 22.78, 28.73, 10.64]


[10.85, 25.79, 10.8, 27.04, 41.46, 26.65, 51.39, 26.53, 55.09, 40.51, 50.71, 47.67, 28.01, 29.88, 52.28, 23.56, 37.41, 22.01, 39.88, 22.81, 28.76, 45.81, 10.65]
[10.82, 25.7, 10.77, 26.95, 41.32, 26.57, 51.22, 48.4, 46.34, 40.38, 50.54, 40.09, 27.92, 29.78, 52.11, 23.48, 37.29, 21.94, 39.75, 22.74, 28.67, 109.53, 46.96, 10.61]
[10.85, 25.78, 10.8, 27.03, 41.45, 26.65, 115.24, 51.38, 26.53, 33.24, 40.51, 50.7, 28.76, 28.0, 29.87, 52.27, 23.56, 64.27, 37.81, 39.87, 22.81, 28.75, 33.57, 18.11, 10.65]
[10.85, 25.79, 10.8, 27.04, 41.46, 26.65, 42.4, 51.39, 26.53, 32.06, 40.51, 50.71, 11.47, 47.67, 28.01, 51.34, 52.28, 23.56, 37.41, 22.01, 39.88, 22.81, 28.76, 32.38, 16.85, 10.65]
[10.81, 25.69, 10.76, 26.93, 41.3, 26.55, 42.24, 51.19, 26.43, 31.94, 40.36, 50.51, 10.19, 44.44, 27.9, 47.86, 52.08, 23.47, 37.26, 21.92, 39.73, 22.72, 28.65, 32.25, 16.79, 18.23, 51.76]
[10.79, 25.63, 10.74, 26.88, 41.21, 26.49, 42.15, 51.08, 26.37, 31.87, 40.27, 50.4, 9.18, 41.85, 27.84, 45.07, 51.97, 23.42, 37.

[10.82, 51.25, 25.72, 10.77, 26.96, 41.34, 26.58, 42.29, 51.25, 26.46, 31.97, 40.4, 50.57, 8.4, 39.88, 27.93, 42.95, 52.14, 23.5, 37.31, 21.95, 39.77, 22.75, 28.68, 32.29, 16.81, 45.9, 10.62, 19.06]
[10.81, 30.66, 25.69, 10.76, 26.93, 41.29, 26.55, 42.23, 51.18, 26.43, 31.93, 40.35, 50.5, 7.72, 38.02, 27.9, 40.94, 52.08, 23.47, 37.26, 21.92, 39.72, 22.72, 28.64, 104.06, 32.25, 16.78, 45.85, 10.61, 19.04]
[65.18, 10.83, 22.6, 25.73, 10.77, 26.97, 41.36, 26.59, 42.3, 51.26, 26.47, 31.98, 40.41, 50.58, 7.15, 36.5, 27.94, 39.31, 52.16, 23.5, 37.32, 21.95, 39.78, 22.75, 28.69, 38.34, 32.3, 16.81, 45.92, 10.62, 19.07]
[33.96, 10.82, 18.86, 25.72, 10.77, 26.96, 41.34, 26.58, 42.29, 51.25, 26.46, 31.97, 40.4, 50.57, 6.66, 35.11, 27.93, 37.81, 57.42, 89.59, 40.37, 37.31, 21.95, 39.77, 22.75, 28.68, 38.33, 32.29, 16.81, 45.9, 10.62, 19.06]
[24.24, 10.85, 18.9, 25.78, 10.8, 27.03, 71.21, 26.64, 42.39, 51.37, 26.52, 32.05, 40.5, 50.69, 6.24, 33.96, 28.0, 36.57, 32.41, 65.15, 29.36, 37.39, 22.0, 39

[23.96, 10.82, 18.85, 25.71, 10.77, 104.02, 26.95, 71.01, 26.57, 42.27, 51.23, 26.45, 31.96, 40.38, 50.54, 5.52, 31.77, 27.92, 34.21, 37.57, 21.11, 52.12, 23.49, 37.29, 21.94, 39.75, 39.07, 28.67, 38.31, 32.27, 16.8, 78.84, 10.61, 19.05]
[23.96, 18.59, 18.85, 25.71, 10.77, 38.27, 26.96, 71.02, 26.57, 42.28, 51.24, 26.45, 126.97, 54.92, 40.39, 50.55, 5.22, 52.12, 27.92, 33.25, 37.58, 21.12, 52.13, 23.49, 37.3, 21.94, 39.76, 22.74, 28.67, 38.32, 32.28, 28.87, 45.89, 10.62, 19.06]
[23.97, 10.82, 18.85, 25.72, 10.77, 38.28, 46.33, 71.03, 26.58, 42.28, 51.24, 26.46, 46.72, 31.97, 40.4, 50.56, 4.96, 30.06, 27.93, 32.37, 37.58, 21.12, 52.14, 23.49, 64.1, 74.06, 21.95, 39.77, 22.75, 28.68, 38.32, 32.28, 16.8, 45.9, 10.62, 19.06]
[23.95, 10.82, 18.84, 25.7, 10.76, 38.26, 30.76, 70.99, 26.56, 72.61, 51.21, 26.44, 46.69, 31.95, 40.37, 50.53, 4.71, 29.29, 27.91, 31.54, 37.56, 21.11, 52.1, 23.48, 42.56, 35.29, 21.93, 39.74, 22.73, 28.66, 112.44, 38.3, 32.26, 16.79, 45.87, 10.61, 19.05]
[23.9, 10.79

68×68 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱                      ⋮         
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  